In [1]:
import numpy as np
import copy
import matplotlib.pyplot as plt
import sklearn
import sklearn.datasets
import sklearn.linear_model
from PIL import Image
import os
from tqdm import tqdm
from sklearn.utils import shuffle 
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
import tensorflow as tf

import warnings
warnings.filterwarnings("ignore")

In [3]:
# Verimizde iki tür sınıf vardır kedi ve köpek
labels = ["dogs","cats"]

X_train = []
y_train = []

X_test = []
y_test = []

image_size = 28

# Bu adımda görüntü veri kümesini eğitim (training_set) ve test (test_set) olarak ikiye ayırarak yükler.
# Veriler, data/ klasörünün içindeki training_set/ ve test_set/ dizinlerinden okunur.
for label in labels:
    folderPath = os.path.join("..","data","training_set", label)
    for file in tqdm(os.listdir(folderPath)):
        
        img = Image.open(os.path.join(folderPath, file))
        img = img.resize((image_size, image_size))
      
        X_train.append(img)
        y_train.append(label)
        
for label in labels:
    folderPath = os.path.join("..","data","test_set", label)
    for file in tqdm(os.listdir(folderPath)):
        
        img = Image.open(os.path.join(folderPath, file))
        img = img.resize((image_size, image_size))
      
        X_test.append(img)
        y_test.append(label)

100%|██████████| 1011/1011 [00:20<00:00, 48.67it/s]


In [4]:



# Okunan görüntüler numpy tipine çevrilir
# Her işlem train ve test setleri için gerçekleştirilir
X_train = np.array(X_train).reshape(-1,image_size*image_size*3)
X_train = X_train/255.0
y_train = np.array(y_train)

y_train_new = []

for i in y_train:
    y_train_new.append(labels.index(i))
y_train = y_train_new
y_train = np.array(y_train)
y_train = np.array(y_train).reshape(-1, 1)
y_train = tf.keras.utils.to_categorical(y_train)


X_test = np.array(X_test).reshape(-1,image_size*image_size*3)
X_test = X_test/255.0
y_test = np.array(y_test)

y_test_new = []

for i in y_test:
    y_test_new.append(labels.index(i))
y_test = y_test_new
y_test = np.array(y_test)
y_test = np.array(y_test).reshape(-1, 1)
y_test = tf.keras.utils.to_categorical(y_test)

    

# Verimiz kedi ve köpek resimlerinin sıralı birşekilde olmasından dolayı rastgele bir şekilde karıştırılır
X_train, y_train = shuffle(X_train, y_train, random_state=42)
X_test, y_test = shuffle(X_test, y_test, random_state=42)

# Son olarak train ve test verilerini modelimize girdi olarak vermek için
# uygun boyutlara getirip veri ön işleme adımlarını tamamlıyoruz
X_train = X_train.T
X_test = X_test.T

In [5]:
y_train = y_train.reshape(2,-1)
y_test = y_test.reshape(2,-1)

In [6]:
def sigmoid(z):
    s = 1/(1+np.exp(-z))
    return s

In [7]:
def katman_boyutlari(X, Y):
    # n_x verideki bir örneğin sahip olduğu özellik sayısı
    # n_h gizli katman boyutu
    # n_y sınıflandırılacak sınıf sayısı

    n_x = X.shape[0]
    n_h = 4
    n_y = Y.shape[0]

    return (n_x, n_h, n_y)

In [8]:
def paramtetre_başlat(n_x, n_h, n_y):

    # parametrelerimizi uygun boyutlarda başlatıyoruz
    W1 = np.random.randn(n_h, n_x)*0.01
    b1 = np.zeros((n_h,1))

    w2 = np.random.randn(n_y, n_h)*0.01
    b2 = np.zeros((n_y, 1))

    parameters = {"W1":W1,
                  "b1":b1,
                  "W2":w2,
                  "b2":b2}
    return parameters

In [9]:
def ileri_yayilim(X, parameters):
    # parametrelerimizi çekiyoruz
    W1 = parameters["W1"]
    b1 = parameters["b1"]
    W2 = parameters["W2"]
    b2 = parameters["b2"]

    Z1 = np.dot(W1,X)+b1
    # tanh fonksiyonu sigmoid fonksiyonundan genel durumlarda daha iyi performans gösterir
    # sigmoid fonksiyonun öne çıktığı tek durum ikili sınıflandırma yapılacağı zamandır 
    A1 = np.tanh(Z1)
    Z2 = np.dot(W2,A1)+b2
    A2 = sigmoid(Z2)

    cache = {"Z1":Z1,
             "A1":A1,
             "Z2":Z2,
             "A2":A2}
    
    return A2, cache

In [10]:
def maliyet_hesaplama(A2, Y):
    # Örnek sayısı
    m = Y.shape[1]

    """
    logprobs = np.multiply(np.log(A2),Y)+np.multiply((1-Y),(np.log(1-A2)))
    cost = -np.sum(logprobs)/m

    cost = float(np.squeeze(cost)) # (1,1) boyutundaki matrisi skaler bir sayıya çevirir

    return cost
    """
    logprobs = np.multiply(np.log(A2),Y)+np.multiply((1-Y),(np.log(1-A2)))
    cost = -np.sum(logprobs)/m
    
    # YOUR CODE ENDS HERE
    
    cost = float(np.squeeze(cost))  # makes sure cost is the dimension we expect. 
                                    # E.g., turns [[17]] into 17 
    
    return cost

In [24]:
def geri_yayilim(parameters, cache, X, Y):

    m = X.shape[1]

    W1 = parameters["W1"]
    W2 = parameters["W2"]

    A1 = cache["A1"]
    A2 = cache["A2"]

    # parametrelerimizin türevlerini alıyoruz
    dZ2 = A2-Y
    dW2 = np.dot(dZ2,A1.T)/m
    db2 = np.sum(dZ2, axis=1, keepdims=True)/m
    dZ1 = np.dot(W2.T, dZ2)*(1-np.power(A1,2))
    dW1 = np.dot(dZ1,X.T)/m
    db1 = np.sum(dZ1, axis=1, keepdims=True)/m

    grads = {"dW1":dW1,
             "db1":db1,
             "dW2":dW2,
             "db2":db2}
    
    return grads

In [28]:
def parametre_güncelleme(parameters, grads, learning_rate=1.2):

    # parametrelerimizi çekiyoruz
    W1 = copy.deepcopy(parameters["W1"])
    b1 = parameters["b1"]
    W2 = copy.deepcopy(parameters["W2"])
    b2 = parameters["b2"]
    
    dW1 = grads["dW1"]
    db1 = grads["db1"]
    dW2 = grads["dW2"]
    db2 = grads["db2"]

    # verilen learning_rate değeriyle parametrelerimizi güncelliyoruz
    W1 = W1 - learning_rate*dW1
    b1 = b1 - learning_rate*db1
    W2 = W2 - learning_rate*dW2
    b2 = b2 - learning_rate*db2

    parameters = {"W1":W1,
                  "b1":b1,
                  "W2":W2,
                  "b2":b2}
    
    return parameters


In [30]:
# GRADED FUNCTION: predict

def predict(parameters, X):
    """
    Using the learned parameters, predicts a class for each example in X
    
    Arguments:
    parameters -- python dictionary containing your parameters 
    X -- input data of size (n_x, m)
    
    Returns
    predictions -- vector of predictions of our model (red: 0 / blue: 1)
    """
    
    # Computes probabilities using forward propagation, and classifies to 0/1 using 0.5 as the threshold.
    #(≈ 2 lines of code)
    # A2, cache = ...
    # predictions = ...
    # YOUR CODE STARTS HERE
    
    A2, cache = ileri_yayilim(X, parameters)
    predictions = (A2 >= 0.5).astype(int)
    
    # YOUR CODE ENDS HERE
    
    return predictions

In [35]:
def nn_model(X_train, Y_train, X_test, Y_test, n_h, num_iterations=10000, learning_rate=0.001, print_cost=False):

    # oluşturduğumuz fonksiyonları tek bir model altında birleştirelim
    np.random.seed(3)
    n_x = katman_boyutlari(X_train, Y_train)[0]
    n_y = katman_boyutlari(X_train, Y_train)[2]

    parameters = paramtetre_başlat(n_x, n_h, n_y)

    # verilen num_iterations değeri kadar döngü işlemi uygulicaz
    for i in range(num_iterations):

        A2, cache = ileri_yayilim(X_train, parameters)
        cost = maliyet_hesaplama(A2, Y_train)
        grads = geri_yayilim(parameters, cache, X_train, Y_train)
        parameters = parametre_güncelleme(parameters, grads, learning_rate)
        
        if print_cost and i % 100 == 0:
            print ("Cost after iteration %i: %f" %(i, cost))
            if print_cost:
                Y_prediction_train = predict(parameters, X_train)
                Y_prediction_test = predict(parameters, X_test)
                print("train accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_train - Y_train)) * 100))
                print("test accuracy: {} %".format(100 - np.mean(np.abs(Y_prediction_test - Y_test)) * 100))
 
    return parameters

In [38]:
nn_model(X_train, y_train, X_test, y_test, n_h=7, num_iterations=2000, print_cost=True)

Cost after iteration 0: 1.386290
train accuracy: 49.95627732667083 %
test accuracy: 50.14829461196243 %
Cost after iteration 100: 1.386289
train accuracy: 49.98750780762023 %
test accuracy: 50.02471576866041 %
Cost after iteration 200: 1.386289
train accuracy: 50.01249219237977 %
test accuracy: 50.0 %
Cost after iteration 300: 1.386288
train accuracy: 50.10618363522798 %
test accuracy: 49.87642115669797 %
Cost after iteration 400: 1.386288
train accuracy: 50.118675827607746 %
test accuracy: 49.85170538803757 %
Cost after iteration 500: 1.386287
train accuracy: 50.15615240474703 %
test accuracy: 49.90113692535838 %
Cost after iteration 600: 1.386287
train accuracy: 50.12492192379763 %
test accuracy: 49.82698961937716 %
Cost after iteration 700: 1.386286
train accuracy: 50.199875078076204 %
test accuracy: 49.80227385071676 %
Cost after iteration 800: 1.386285
train accuracy: 50.20612117426608 %
test accuracy: 49.70341077607514 %
Cost after iteration 900: 1.386285
train accuracy: 50.25608

KeyboardInterrupt: 